In [11]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import glm
import statsmodels.api as sm

# Load the training data
train_data = pd.read_csv('InsNova_data_2023_train.csv/InsNova_data_2023_train.csv')

# Check mean and variance of numclaims
mean_numclaims = train_data['numclaims'].mean()
var_numclaims = train_data['numclaims'].var()

print(f"Mean of numclaims: {mean_numclaims}")
print(f"Variance of numclaims: {var_numclaims}")

# # Poisson regression model for Frequency
# formula = 'numclaims ~ np.log(exposure) + agecat + area + veh_value + veh_age + veh_value:veh_age + area:veh_value'
# frequency_model = glm(formula, data=train_data, family=sm.families.Poisson()).fit()

formula = 'numclaims ~  exposure + veh_value + agecat + veh_age + gender + credit_score '
frequency_model = glm(formula, data=train_data, family = sm.families.Poisson()).fit()

# formula = 'numclaims ~  exposure + high_education_ind + agecat + credit_score '
# frequency_model = glm(formula, data=train_data).fit()


print(frequency_model.summary())


Mean of numclaims: 0.07312436447234626
Variance of numclaims: 0.0784796313185289
                 Generalized Linear Model Regression Results                  
Dep. Variable:              numclaims   No. Observations:                22619
Model:                            GLM   Df Residuals:                    22612
Model Family:                 Poisson   Df Model:                            6
Link Function:                    Log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -5886.6
Date:                Thu, 16 Nov 2023   Deviance:                       8622.7
Time:                        22:27:30   Pearson chi2:                 2.35e+04
No. Iterations:                     6   Pseudo R-squ. (CS):            0.01534
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------

In [15]:
# Inverse Gaussian model for Severity
# Create a new column for the average claim cost
#train_data_with_claims['avg_claim_cost'] = train_data_with_claims['claimcst0'] / train_data_with_claims['numclaims']

#Inverse Gaussian model for Severity
# severity_model = glm('avg_claim_cost ~ gender + veh_age + agecat', 
#                     data=train_data, 
#                     family=sm.families.InverseGaussian(link=sm.families.links.log())).fit()

train_data['avg_claim_cost'] = train_data['claimcst0'] / train_data['numclaims']

severity_model = glm('avg_claim_cost ~ gender + veh_age + agecat + veh_value + exposure + credit_score ', 
                     data=train_data, family=sm.families.InverseGaussian(link=sm.families.links.log())).fit()

# severity_model = glm('avg_claim_cost ~  credit_score + max_power + veh_age', 
#                      data=train_data).fit()

print(severity_model.summary())
""


                 Generalized Linear Model Regression Results                  
Dep. Variable:         avg_claim_cost   No. Observations:                 1542
Model:                            GLM   Df Residuals:                     1535
Model Family:         InverseGaussian   Df Model:                            6
Link Function:                    log   Scale:                       0.0013533
Method:                          IRLS   Log-Likelihood:                -12999.
Date:                Thu, 16 Nov 2023   Deviance:                       1.9769
Time:                        22:30:17   Pearson chi2:                     2.08
No. Iterations:                    34   Pseudo R-squ. (CS):            0.01214
Covariance Type:            nonrobust                                         
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        8.6544      2.515      3.441   

''

In [16]:
from sklearn.model_selection import KFold
import numpy as np

def gini(actual, pred):
    assert len(actual) == len(pred)
    all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
    all = all[np.lexsort((all[:, 2], -1 * all[:, 1]))]
    total_losses = all[:, 0].sum()
    gini_sum = all[:, 0].cumsum().sum() / total_losses
    gini_sum -= (len(actual) + 1) / 2.
    return gini_sum / len(actual)

def normalized_gini(actual, pred):
    return gini(actual, pred) / gini(actual, actual)

# Prepare for cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=1)
gini_scores = []

for train_index, test_index in kf.split(train_data):
    cv_train, cv_test = train_data.iloc[train_index], train_data.iloc[test_index]
    
    # Fit the model on cv_train
    #cv_model = glm(formula, data=cv_train, family=sm.families.Poisson()).fit()
    cv_model = glm(formula, data=cv_train).fit()
    # Predict on cv_test
    predictions = cv_model.predict(cv_test)
    
    # Compute and store Gini coefficient
    gini_score = normalized_gini(cv_test['numclaims'], predictions)
    gini_scores.append(gini_score)

# Calculate the average Gini coefficient
average_gini = np.mean(gini_scores)
print(f"Average Gini Coefficient from Cross-Validation: {average_gini}")


C:\Users\ryane\AppData\Local\Temp\ipykernel_18316\1342160008.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)
C:\Users\ryane\AppData\Local\Temp\ipykernel_18316\1342160008.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred,

Average Gini Coefficient from Cross-Validation: 0.28904236004363915


C:\Users\ryane\AppData\Local\Temp\ipykernel_18316\1342160008.py:6: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  all = np.asarray(np.c_[actual, pred, np.arange(len(actual))], dtype=np.float)


In [19]:
print(f"Average Gini Coefficient from Cross-Validation: {average_gini}")

Average Gini Coefficient from Cross-Validation: 0.28904236004363915


In [20]:
import random

def bootstrap_model(data, model_formula, family, R=500):
    boot_coefs = []

    for _ in range(R):
        boot_sample = data.sample(frac=1, replace=True, random_state=random.randint(1, 10000))
        boot_model = glm(model_formula, data=boot_sample, family=family).fit()
        boot_coefs.append(boot_model.params)

    # Convert list of coefficients to DataFrame
    boot_coefs_df = pd.DataFrame(boot_coefs)

    # Calculate the average coefficients
    avg_coefs = boot_coefs_df.mean(axis=0)
    return avg_coefs

# Apply bootstrapping
avg_boot_coefs = bootstrap_model(train_data, formula, sm.families.Poisson(), R=500)
print(avg_boot_coefs)


Intercept      -2.322846
gender[T.M]    -0.077269
exposure        1.569166
veh_value       0.008341
agecat         -0.088220
veh_age         0.008628
credit_score   -0.001194
dtype: float64


In [21]:
# Load the vh data
vh_data = pd.read_csv('InsNova_data_2023_vh.csv/InsNova_data_2023_vh.csv')
# Compute Frequency and Severity predictions
# (Replace `frequency_model` and `severity_model` with your actual model objects)
vh_data['predicted_frequency'] = frequency_model.predict(vh_data)
vh_data['predicted_severity'] = severity_model.predict(vh_data)

# Calculate the final claim cost predictions
vh_data['predicted_claimcst0'] = vh_data['predicted_frequency'] * vh_data['predicted_severity']
# Prepare the output DataFrame
output = vh_data[['id', 'predicted_claimcst0']]
output.rename(columns={'predicted_claimcst0': 'Predict'}, inplace=True)
# Export to CSV
# Assigning 0 to negative values of Predict
output.loc[output['Predict'] < 0, 'Predict'] = 0
output.to_csv('final_predictions7.csv', index=False)



C:\Users\ryane\AppData\Local\Temp\ipykernel_18316\4054090848.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  output.rename(columns={'predicted_claimcst0': 'Predict'}, inplace=True)
